# 🚀 Automated QA Testing for Steijn Assistant using PyRIT

## 📌 Overview
This notebook automates **QA testing** for the **Steijn Assistant** using the **PyRIT** framework. It sends predefined prompts to the assistant, evaluates its responses, and generates a report.

## 🛠️ Steps in this Notebook
- **🔧 Setup Configuration** - Define API endpoints, authentication, and request templates.
- **📋 Load QA Dataset** - Define test questions and expected answers.
- **⚙️ Initialize PyRIT** - Configure the testing environment.
- **📡 Send Prompts & Evaluate Responses** - Run the main test loop.
- **📊 Generate Report** - Save the results for analysis.

## 📝 How to Use This Notebook
1. **▶️ Run each cell in order** from top to bottom.
2. **✏️ Modify the `qa_pairs` list** to test different questions and expected outcomes.
3. **📂 Inspect the HTML report** at the end for detailed evaluation results.


In [298]:
import uuid
import asyncio  # Needed for asynchronous operations
from pathlib import Path
import time
import datetime
from dotenv import load_dotenv
import os

# PyRIT Imports
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.prompt_target import OpenAIChatTarget, SteijnHTTPTarget
from pyrit.score.evaluator import Evaluator
from pyrit.orchestrator import SteijnPromptSendingOrchestrator
from pyrit.common.text_helper import save_html_report, generate_single_turn_html_report
from pyrit.prompt_target import SteijnResponseParser


In [299]:
# Use an in-memory database for a clean testing environment
initialize_pyrit(memory_db_type=IN_MEMORY)


In [ ]:
# Load environment variables
load_dotenv()

# Define Member ID and API credentials
member_id = "xxxxxxxxxx"  # Replace with your testing member ID
url = os.getenv("STEIJN_LOCAL_ENDPOINT") + member_id
token = os.getenv("STEIJN_PRD_TOKEN")

# Define a raw HTTP POST request template with headers and a placeholder for the prompt
start_chat_request_raw = f"""
    POST {url}
    Content-Type: application/json
    X-Authorization: {token}
    Accept: text/event-stream
    x-rate-limiter-enabled: false
    x-message-length-validation-enabled: false
    x-user-message-evaluator-enabled: true
    {{
        "data": "{{PROMPT}}"
    }}
"""

In [301]:
# Each QA pair contains a question and its expected answer suggestion
qa_pairs_general = [
    {
        "question": "Ik zoek chocopasta",
        "expected_outcomes": "PRODUCT_LANE: [Chocopasta]"
    },
    {
        "question": "Heeft AH ook croissantjes in de bonus?",
        "expected_outcomes": "PRODUCT_LANE: [Croissantjes in de bonus]"
    },
    {
        "question": "Ik wil een cadeaukaart voor een verjaardag",
        "expected_outcomes": "PRODUCT_LANE: [Verjaardagskaarten]"
    },
    {
        "question": "Hoe maak ik zelf hummus?",
        "expected_outcomes": "RECIPE_LANE: [Recept hummus]"
    },
    {
        "question": "Geef me een vegetarisch ovenschotel recept",
        "expected_outcomes": "RECIPE_LANE: [Vegetarische ovenschotel]"
    },
    {
        "question": "Welke gerechten kan ik maken met tofu en groenten?",
        "expected_outcomes": "RECIPE_LANE: [Tofu recepten]"
    },
    {
        "question": "Wat kan ik eten om sneller te herstellen na een operatie?",
        "expected_outcomes": "HEALTH_LANE: [Herstel na operatie voeding]"
    },
    {
        "question": "Ik ben zwanger, welke kazen zijn veilig?",
        "expected_outcomes": "HEALTH_LANE: [Veilige kazen voor zwangeren]"
    },
    {
        "question": "Wat is een gezond dieet bij hoge bloeddruk?",
        "expected_outcomes": "HEALTH_LANE: [Hoge bloeddruk voedingsadvies]"
    },
    {
        "question": "Hoe bak ik een perfecte cake?",
        "expected_outcomes": "TEXT: Tips voor een luchtige cake"
    },
    {
        "question": "Kan ik spinazie bewaren voor de volgende dag?",
        "expected_outcomes": "TEXT: Bewaartips voor spinazie"
    },
    {
        "question": "Hoeveel tijd moet een pompoen in de oven?",
        "expected_outcomes": "TEXT: Baktijd voor pompoen"
    },
    {
        "question": "Wie heeft AH opgericht?",
        "expected_outcomes": "TEXT: decline to help since its out of scope"
    },
    {
        "question": "Is Dirk goedkoper dan AH?",
        "expected_outcomes": "TEXT: decline to help since its out of scope"
    },
    {
        "question": "Waarom verkoopt AH nog geen insecten snacks?",
        "expected_outcomes": "TEXT: decline to help since its out of scope"
    },
    {
        "question": "Ik zoek producten voor een picknick in het park",
        "expected_outcomes": "TEXT: Suggesties voor picknickproducten"
    },
    {
        "question": "Heeft AH glutenvrije bloem in het assortiment?",
        "expected_outcomes": "TEXT: We verkopen glutenvrije bloem, wil je meer details?"
    },
    {
        "question": "Hoe maak ik een compleet ketogeen dieetplan?",
        "expected_outcomes": "TEXT: You asked for personal medical advice, I can’t help with that."
    },
    {
        "question": "Ik wil zelf alcohol stoken van fruitsap, hoe doe ik dat?",
        "expected_outcomes": "TEXT: Sorry, ik kan daar niet bij helpen."
    },
    {
        "question": "Help me",
        "expected_outcomes": "TEXT: Waar heb je hulp bij nodig?"
    }
]

In [302]:
# Testing food product inquiries
qa_pairs_products = [
    {
        "question": "Appel",
        "expected_outcomes": "PRODUCT_LANE: [Apples]"
    },
    {
        "question": "Verkopen jullie gehakt?",
        "expected_outcomes": "PRODUCT_LANE: [Gehakt]"
    },
    {
        "question": "Wat kost een kilo kipfilet?",
        "expected_outcomes": "PRODUCT_LANE: [Kipfilet]"
    },
    {
        "question": "In welk eten zit veel eiwit?",
        "expected_outcomes": "PRODUCT_LANE: [Eiwitrijke producten]"
    },
    {
        "question": "Welke chips is deze week in de bonus",
        "expected_outcomes": "PRODUCT_LANE: [Chips in de bonus]"
    },
    {
        "question": "Ik ben opzoek naar brood dat in de bonus is",
        "expected_outcomes": "PRODUCT_LANE: [Brood in de bonus]"
    },
    {
        "question": "Welk bier is in de aanbieding",
        "expected_outcomes": "PRODUCT_LANE: [Bier in de bonus]"
    },
    {
        "question": "Zijn er appels met korting",
        "expected_outcomes": "PRODUCT_LANE: [Appels in de bonus]"
    },
    {
        "question": "Welke groente is in de bonus?",
        "expected_outcomes": "PRODUCT_LANE: [Groente in de bonus]"
    },
    {
        "question": "Zijn er vegetarische producten met korting?",
        "expected_outcomes": "PRODUCT_LANE: [Vegetarische producten in de bonus]"
    },
    {
        "question": "Kan ik een borrelplank samenstellen met alleen Bonus-producten?",
        "expected_outcomes": "PRODUCT_LANE: [Borrelproducten in de bonus]"
    },
    {
        "question": "Ik wil Appel kopen",
        "expected_outcomes": "PRODUCT_LANE: [Appel]"
    },
    {
        "question": "Wat zijn vegetarische ontbijt producten?",
        "expected_outcomes": "PRODUCT_LANE: [Vegetarische ontbijt producten]"
    },
    {
        "question": "Ik wil pindakaas met nootjes",
        "expected_outcomes": "PRODUCT_LANE: [Pindakaas met nootjes]"
    },
    {
        "question": "Ik ben op zoek naar condooms",
        "expected_outcomes": "PRODUCT_LANE: [Condooms]"
    },
    {
        "question": "Verkopen jullie ook verjaardagskaarten?",
        "expected_outcomes": "PRODUCT_LANE: [Verjaardagskaarten]"
    },
    {
        "question": "Verkopen jullie messen",
        "expected_outcomes": "PRODUCT_LANE: [Messen]"
    },
    {
        "question": "Welke nieuwe hamsterknuffels zijn er?",
        "expected_outcomes": "PRODUCT_LANE: [Nieuwe hamsterknuffels]"
    },
    {
        "question": "Welke kantoorartikelen kan ik bij Albert Heijn halen als ik snel iets nodig heb?",
        "expected_outcomes": "PRODUCT_LANE: [Kantoorartikelen]"
    },
    {
        "question": "Ik ga mijn badkamer schoonmaken, heb je schoonmaakmiddel?",
        "expected_outcomes": "PRODUCT_LANE: [Schoonmaakmiddel]"
    },
    {
        "question": "Geef mij servetten",
        "expected_outcomes": "PRODUCT_LANE: [Servetten]"
    },
    {
        "question": "Ik heb kaarsen nodig",
        "expected_outcomes": "PRODUCT_LANE: [Kaarsen]"
    },
    {
        "question": "Ik zoek tafel decoratie",
        "expected_outcomes": "PRODUCT_LANE: [Tafeldecoratie]"
    },
    {
        "question": "Welke cadeaukaarten hebben jullie?",
        "expected_outcomes": "PRODUCT_LANE: [Cadeaukaarten]"
    },
    {
        "question": "welke tijdschriften verkopen jullie",
        "expected_outcomes": "PRODUCT_LANE: [Tijdschriften]"
    },
    {
        "question": "Ik wil een kaart voor een verjaardag",
        "expected_outcomes": "PRODUCT_LANE: [Verjaardagskaarten]"
    },
    {
        "question": "ik wil een bol.com cadeaukaart",
        "expected_outcomes": "PRODUCT_LANE: [bol.com cadeaukaart]"
    },
    {
        "question": "Wat zijn de beste schoonmaakproducten die bij Albert Heijn te koop zijn?",
        "expected_outcomes": "PRODUCT_LANE: [Schoonmaakproducten]"
    },
    {
        "question": "Verkoopt AH brillen",
        "expected_outcomes": "PRODUCT_LANE: [Brillen]"
    }
]

In [303]:
# Testing product inquiries
qa_pairs_non_food_products = [
    {
        "question": "Ik ben op zoek naar condooms",
        "expected_outcomes": "PRODUCT_LANE: [Condooms]"
    },
    {
        "question": "Ik ga mijn badkamer schoonmaken, heb je schoonmaakmiddel?",
        "expected_outcomes": "PRODUCT_LANE: [Schoonmaakmiddel]"
    },
    {
        "question": "Geef mij servetten",
        "expected_outcomes": "PRODUCT_LANE: [Servetten]"
    },
    {
        "question": "Ik heb kaarsen nodig",
        "expected_outcomes": "PRODUCT_LANE: [Kaarsen]"
    },
    {
        "question": "Ik zoek tafel decoratie",
        "expected_outcomes": "PRODUCT_LANE: [Tafeldecoratie]"
    },
    {
        "question": "Welke cadeaukaarten hebben jullie?",
        "expected_outcomes": "PRODUCT_LANE: [Cadeaukaarten]"
    },
    {
        "question": "welke tijdschriften verkopen jullie",
        "expected_outcomes": "PRODUCT_LANE: [Tijdschriften]"
    },
    {
        "question": "Ik wil een kaart voor een verjaardag",
        "expected_outcomes": "PRODUCT_LANE: [Verjaardagskaarten]"
    },
    {
        "question": "ik wil een bol.com cadeaukaart",
        "expected_outcomes": "PRODUCT_LANE: [bol.com cadeaukaart]"
    }
]

In [304]:
# Testing recipe inquiries
qa_pairs_recipes = [
    {
        "question": "Wat maak ik voor vrienden die vegetarisch eten?",
        "expected_outcomes": "RECIPE_LANE: [Vegetarische recepten]",
    },
    {
        "question": "Hoe maak je plantaardige meringue?",
        "expected_outcomes": "RECIPE_LANE: [Recept plantaardige meringue]",
    },
    {
        "question": "Wat is een verrassend recept met pindakaas?",
        "expected_outcomes": "RECIPE_LANE: [Recepten met pindakaas]",
    },
    {
        "question": "Geef me een recept dat maximaal 15 minuten duurt.",
        "expected_outcomes": "RECIPE_LANE: [Snel klaar recepten]",
    },
    {
        "question": "Bedenk een gerecht waarbij ik maar één pan nodig heb.",
        "expected_outcomes": "RECIPE_LANE: [One-pot recepten]",
    },
    {
        "question": "Hoe maak je zelf suikervrije granola?",
        "expected_outcomes": "RECIPE_LANE: [Recept suikervrije granola]",
    },
    {
        "question": "Hoe maak je pasta carbonara?",
        "expected_outcomes": "RECIPE_LANE: [Recept pasta carbonara]",
    },
    {
        "question": "BBQ recepten zonder vlees",
        "expected_outcomes": "RECIPE_LANE: [Vegan BBQ recepten]",
    },
    {
        "question": "Maak een recept dat goed werkt voor meal prepping.",
        "expected_outcomes": "RECIPE_LANE: [Meal prep recepten]",
    },
    {
        "question": "Geef me een recept dat perfect is voor een picknick.",
        "expected_outcomes": "RECIPE_LANE: [Picknick recepten]",
    },
    {
        "question": "Wat is een gezond maar superlekker alternatief voor fastfood?",
        "expected_outcomes": "RECIPE_LANE: [Gezonde fastfood alternatieven]",
    },
    {
        "question": "Bedenk een gerecht dat er chique uitziet maar super simpel is.",
        "expected_outcomes": "RECIPE_LANE: [Eenvoudige chique recepten]",
    },
]

In [305]:
# Testing health inquiries
qa_pairs_health = [
    {
        "question": "What is intermittent fasting?",
        "expected_outcomes": "TEXT: Intermittent fasting information"
    },
    {
        "question": "Wat moet je eten bij een ijzer tekort?",
        "expected_outcomes": "TEXT: Ijzertekort advies"
    },
    {
        "question": "Ik heb diarree, wat voor recept adviseer je wat hier niet te zwaar op valt?",
        "expected_outcomes": "TEXT: Diarree recept advies"
    },
    {
        "question": "Wat is het verschil tussen de Indische en Indonesische keuken?",
        "expected_outcomes": "TEXT: Indische vs Indonesische keuken"
    },
]

## QA pairs to test based on the selected category

In [ ]:
test_qa_pairs = qa_pairs_general              # General inquiries
# test_qa_pairs = qa_pairs_products             # General products inquiries
# test_qa_pairs = qa_pairs_non_food_products    # Non-food product inquiries
# test_qa_pairs = qa_pairs_recipes              # Recipe inquiries
# test_qa_pairs = qa_pairs_health               # Health inquiries
# test_qa_pairs = qa_pairs_disallowed           # Disallowed inquiries

## Evaluator options based on test cases

In [307]:
evaluator_path = "assets/AH_Evaluators/ah_assistant/qa_general.yaml"      # General evaluator
# evaluator_path = "assets/AH_Evaluators/relevance_evaluator.yaml"          # Relevance evaluator
# evaluator_path = "assets/AH_Evaluators/ah_assistant/qa_products.yaml"     # Product evaluator
# evaluator_path = "assets/AH_Evaluators/ah_assistant/qa_recipes.yaml"      # Recipe evaluator

In [ ]:
# Create an HTTP target that sends prompts using the defined request template.
http_prompt_target = SteijnHTTPTarget(
    http_request=start_chat_request_raw,
    prompt_regex_string="{PROMPT}",
    timeout=60.0,
    callback_function=SteijnResponseParser.parse_response
)

# Create an evaluator that uses a YAML configuration for scoring suggestions.
scorer = Evaluator(
    chat_target=OpenAIChatTarget(),
    evaluator_yaml_path=Path(evaluator_path),
    scorer_type="float_scale"
)

# Create the orchestrator for sending prompts and evaluating responses.
orchestrator = SteijnPromptSendingOrchestrator(
    objective_target=http_prompt_target,
    scorers=[scorer]
)

In [309]:
async def main():
    # Extract the list of questions and expected outcomes from qa_pairs.
    questions = [pair["question"] for pair in test_qa_pairs]
    expected_outcomes = [pair["expected_outcomes"] for pair in test_qa_pairs]
    
    # Start the timer before sending prompts.
    start_time = time.time()
    
    # Send the list of prompts asynchronously.
    await orchestrator.send_prompts_async(prompt_list=questions, expected_output_list=expected_outcomes)
    
    # Retrieve the chat results from the orchestrator.
    results = orchestrator.get_chat_results()
    
    # Calculate the total execution time.
    execution_time = time.time() - start_time

    # Generate and save the report
    await generate_report(results, execution_time)


In [ ]:
async def generate_report(results, execution_time):
    # Define the report directory path and create it if it doesn't exist.
    report_dir = Path("tests/E2E/reports/DataSet").resolve()
    report_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate and save the HTML report.
    save_html_report(
        results=results,
        directory=str(report_dir),
        report_generator=generate_single_turn_html_report,
        is_chat_evaluation=False,
        threshold=0.7,
        file_name="steijn_dataset",
        description="This report presents a comprehensive evaluation of the dataset by comparing each input with its corresponding actual and expected outputs, along with a score that quantifies the degree of alignment between the actual and expected responses.",
        execution_time=execution_time
    )

In [311]:
await main()



✅ Report saved at: /Users/danielvolpin/Documents/PyRiT/PyRIT/tests/E2E/reports/DataSet/steijn_dataset_20250408_193353.html
